In [2]:
from csv import reader, QUOTE_NONE
from pandas import set_option, read_excel, read_csv, merge
from numpy import array
from sklearn.metrics import cohen_kappa_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
set_option('display.max_columns', None)

columns = [
    'Group1_Tech_Annotations',
    'Group1_Politics_Annotations',
    'Group2_Tech_Annotations',
    'Group2_Politics_Annotations',
    'parent_id',
    'Body'
]

def convert_excel_to_csv():
    group1_df = read_excel('group1_annotations.xlsx').rename(
        columns = {
        'Tech Related': columns[0],
        'Politics Related': columns[1],
        }
    )
    group1_df.to_csv('annotations1.csv', index=False)
    # print(group1_df)

    group2_df = read_excel('group2_annotations.xlsx').rename(
        columns = {
        'Tech Related': columns[2],
        'Politics Related': columns[3],
        }
    ).sort_values('parent_id')
    print(group2_df)
    group2_df.to_csv('annotations2.csv', index=False)

convert_excel_to_csv()


converted_df_to_list = []
group1_df = read_csv('annotations1.csv')
# for idx, row in group1_df.iterrows():
    # print(row[columns[0]].strip)
    # row[columns[2]],
    # row[columns[1]],
    # row[columns[3]],

print(group1_df['Body'].count())
group2_df = read_csv('annotations2.csv')
print(group2_df['Body'].count())
merged_df = merge(group1_df, group2_df, on=[columns[4], columns[5]])
print(merged_df['Body'].count())
merged_df.to_csv('test.csv', index=False)
merged_df[columns[0]] = merged_df[columns[0]].str.upper()
merged_df[columns[2]] = merged_df[columns[2]].str.upper()
merged_df[columns[1]] = merged_df[columns[1]].str.upper()
merged_df[columns[3]] = merged_df[columns[3]].str.upper()

group1_tech = merged_df[columns[0]].to_numpy(dtype=str)
group2_tech = merged_df[columns[2]].to_numpy(dtype=str)
group1_politics = merged_df[columns[1]].to_numpy(dtype=str)
group2_politics = merged_df[columns[3]].to_numpy(dtype=str)
# print(group1_tech)

reformat_group1_tech = []
for row in group1_tech:
    row = row.strip()
    if row == 'T':
        val = 'Y'
        reformat_group1_tech.append(val)
    elif row == 'nan':
        val = 'Y'
        reformat_group1_tech.append(val)
    else:
        reformat_group1_tech.append(row)
group1_tech = reformat_group1_tech

reformat_group1_politics = []
for row in group1_politics:
    row = row.strip()
    reformat_group1_politics.append(row)
# print(reformat_group1_politics)
group1_politics = reformat_group1_politics 

merged_df[columns[0]] = group1_tech
merged_df[columns[1]] = group1_politics
print(merged_df['Group1_Tech_Annotations'].value_counts())

X_dicts = []
y = []
for idx, row in merged_df.iterrows():
    tmp = {
        'g1_tech': row[columns[0]],
        'g2_tech': row[columns[2]],
        'g1_pol': row[columns[1]],
        'g2_pol': row[columns[3]],
    }
    X_dicts.append(tmp)
    y.append(row[columns[5]])

# print(X_dicts)
vec = DictVectorizer()
X = vec.fit_transform(X_dicts)
y = array(y)
# print(X)
# print(y.shape)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5)
print(X_train.shape, X_val.shape)

clf = SVC(C=.1)
clf = SVC(C=100.)
# clf = SVC(C=10.)
# clf = SVC(C=10000.)
clf.fit(X_train, y_train)
preds = clf.predict(X_val)
print(accuracy_score(y_val, preds))

preds = clf.predict(X_train)
print(accuracy_score(y_train, preds))

count_vec = CountVectorizer()


group1_counts = {}
for r1, r2 in zip(group1_tech, group2_tech):
    group1_counts['r1_'+r1+'_r2_'+r2] = group1_counts.get('r1_'+r1+'_r2_'+r2, 0) + 1
# print(group1_counts)

group2_counts = {}
for r1, r2 in zip(group1_politics, group2_politics):
    group2_counts['r1_'+r1+'_r2_'+r2] = group2_counts.get('r1_'+r1+'_r2_'+r2, 0) + 1
# print(group2_counts)
# print(group1_tech)
# print(merged_df)

print('Cohen kappa tech score: ', cohen_kappa_score(group1_tech, group2_tech))
print('Cohen kappa politics score: ', cohen_kappa_score(group1_politics, group2_politics))


# for idx, row in merged_df.iterrows():
#     row_list = [row[columns[4]], row[columns[5]], row[columns[0]], row[columns[2]], row[columns[1]], row[columns[3]]]
#     converted_df_to_list.append(row_list)
# print(converted_df_to_list)

# X_txt_train = []
# y_train = []
# for row in converted_df_to_list:
#     print(row)
#     X_txt_train.append(row[1])
#     y_train.append(row[2:])

# print(X_txt_train)
# print(y_train)
# print(converted_df_to_list)
# print(df)
# data = open("annotations1.csv")
# read_data = reader(data, delimiter='\t', quoting=QUOTE_NONE)

# for row in read_data:
#     print(row)

       parent_id                                               Body  \
515  26179110254  I've been to most of [these](http://maps.googl...   
691  26228204990  we have a lot of shows coming up so check out ...   
494  26244353104  Cheaper for the tax payer (maybe).  We didn't ...   
264  26251770603  I hear ya. I get home from work and get lost i...   
409  26302259456  My hard drive is encrypted on my computer. I t...   
..           ...                                                ...   
343  42172446460  Westside Assistant Principal here. Itâ€™s isnâ...   
183  42172924981  Dope manðŸ‘ŒðŸ½ Iâ€™ll have a cooler with som...   
475  42172955407  Yeah, where are the 2a nuts now? That kid can ...   
359  42174267356                                      I second this   
526  42174557317  Haha fair enough. So then itâ€™s either illite...   

    Group2_Tech_Annotations Group2_Politics_Annotations  
515                       Y                           N  
691                       N    